In [1]:
import json
import pandas as pd
import requests
from tqdm import tqdm
from datetime import datetime
from collections import defaultdict 

In [2]:
# ETAPE 1 connexion à l'API + récupération offres

def create_url(start_url="http://api.adzuna.com/v1/api/jobs/"):
    """
    concactène l'adresse url à partir des paramètres choisis
    comment améliorer pour que les paramètres soient correctement passés ?
    """
    url_list = []
    for page_number in range(1, 5):
        #start_url = "http://api.adzuna.com/v1/api/jobs/"
        country = "fr"
        part0_url = "/search/"
        number_page = str(page_number) 
        part1_url = "?app_id="
        api_id = "37fe08af"
        part2_url = "&app_key="
        api_key = "17c1369ea8fb68bc48f834aebc0bec53"
        part3_url = "&results_per_page="
        results_per_page = str(1)
        part4_url = "&what_and="
        what_and = "data%20engineer"
        part5_url = "&max_days_old="
        max_days_old = str(30)
        end_url = "&content-type=application/json"
        full_url = f"{start_url}{country}{part0_url}{number_page}{part1_url}{api_id}{part2_url}{api_key}{part3_url}{results_per_page}{part4_url}{what_and}{part5_url}{max_days_old}{end_url}"
        url_list.append(full_url)
    return url_list


def scrape_urls(url):
    """
    lance la requête sur l'api Adzuna pour chacunes des url crééés
    si le status_code est incorrect, il s'affiche et l'url liée s'affiche
    """
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("The request for {} was not successful : status {}".format(url, response.status_code))
        return None
    


In [54]:
class Company:
    def __init__(self):
        self.name = None
        self.sector = None
        self.employees = None
        self.creation_year = None
        self.turnover = None
        self.mean_age = None


class JobOffer:
    def __init__(self):
        self.title = None
        self.id_source = None 
        self.company = None
        self.contract_type = None
        self.location = None
        self.salary = None
        self.remote_type = None
        self.starting_date = None
        self.required_experience = None
        self.education = None
        self.description = None
        self.profil_experience = None
        self.publication_date = None
        self.url_direct_offer = None

    

class GetJobOffer():
    def __init__(self, full_info):
        self.full_info = full_info
        self.job_offer = JobOffer()
        self.company = Company()
        

    def get_job_details(self):
        results = self.full_info['results'][0]
        self.job_offer.title = results["title"] if "title" in results else None 
        self.job_offer.id_source = results["id"] if "id" in results else None
        self.job_offer.location = results["location"]["display_name"] if "location" in results else None
        self.job_offer.company = vars(self.company)
        self.job_offer.contract_type = results["contract_type"] if "contract_type" in results else None
        self.job_offer.contract_type += results["contract_time"] if "contract_time" in results else None
        self.job_offer.salary = results["salary_is_predicted"] if "salary_is_predicted" in results else None
        self.job_offer.description = results["description"] if "description" in results else None
        self.job_offer.publication_date = results["created"][:10] if "created" in results else None
        self.job_offer.url_direct_offer = results["redirect_url"] if "redirect_url" in results else None
        
    
    def get_company_details(self):
        self.company.sector = self.full_info['results'][0]["category"]["tag"]
        self.company.name = self.full_info['results'][0]['company']["display_name"]
        self.company.location = self.full_info['results'][0]['company']["display_name"]


    def combine_job_company(self):
        self.get_company_details()
        self.get_job_details()
        return vars(self.job_offer)


In [55]:
def final_step(jobs_offers):
    jobs_ready_for_export = []
    for offer in jobs_offers:
        getjoboffer = GetJobOffer(offer)
        jobs_ready_for_export.append(getjoboffer.combine_job_company())
    return jobs_ready_for_export
    

In [56]:
url_list = create_url()
jobs_offers = [scrape_urls(url) for url in tqdm(url_list, desc="Accessing API")]
jobs_ready_for_export = final_step(jobs_offers)

df_adzuna_jobs = pd.DataFrame(jobs_ready_for_export)

df_adzuna_jobs.to_csv('output/job_offers_wttj.csv')
df_adzuna_jobs

Accessing API: 100%|█| 4/4 [00:04<00:00,  1.09s/it


,title,id_source,company,contract_type,location,salary,remote_type,starting_date,required_experience,education,description,profil_experience,publication_date,url_direct_offer
0,Data Engineer PySpark,4574739084,"{'name': 'THALES', 'sector': 'unknown', 'emplo...",permanentfull_time,"Toulouse, Haute-Garonne",0,None,None,None,None,QUI SOMMES-NOUS ? Thales propose des systèmes ...,None,2024-02-19,https://www.adzuna.fr/land/ad/4574739084?se=bg...
1,Data Engineer Cloud Data F/H,4563519416,"{'name': 'CGI France', 'sector': 'unknown', 'e...",permanentfull_time,"Paris, Ile-de-France",0,None,None,None,None,"Position Description: Big Data, Data Science, ...",None,2024-02-09,https://www.adzuna.fr/land/ad/4563519416?se=Gr...
2,Data Engineer - Banque Data Factory - Nantes,4567731310,"{'name': 'Groupe SOPRA STERIA', 'sector': 'unk...",permanentfull_time,"Nantes, Loire-Atlantique",0,None,None,None,None,"Company Description Sopra Steria , l'un des le...",None,2024-02-13,https://www.adzuna.fr/land/ad/4567731310?se=Rj...
3,Data Engineer Cloud Data F/H,4563521493,"{'name': 'CGI France', 'sector': 'unknown', 'e...",permanentfull_time,"Toulouse, Haute-Garonne",0,None,None,None,None,"Position Description: Big Data, Data Science, ...",None,2024-02-09,https://www.adzuna.fr/land/ad/4563521493?se=QO...


In [158]:
print(type(jobs_ready_for_export))
print(jobs_ready_for_export)

print(vars(jobs_ready_for_export))

print(len(jobs_ready_for_export))

<class '__main__.JobOffer'>
{'title': 'Tech Lead Data Engineer', 'id_source': '4543673772', 'company': 'AXA en France', 'contract_type': None, 'location': 'Nanterre, Hauts-de-Seine', 'salary': '0', 'remote_type': None, 'starting_date': None, 'required_experience': None, 'education': None, 'description': "Environnement En tant que Tech Lead Data Engineer F/H , vous allez contribuer directement aux projets des directions métier (ex : fraude santé, multiéquipements, pricing IARD, optimisation du lead management, fragilité auto, …) d’AXA France et à la construction du socle technique Big Data. Vous allez intégrer une équipe d'une dizaine de personne composée de Data Engineer et des Tech Lead travaillant en mode Feature Team au sein des tribus métier de la Direction Transformation Digital Tech et DA…", 'qualifications': None, 'publication_date': datetime.datetime(2024, 1, 25, 0, 0), 'url_direct_offer': 'https://www.adzuna.fr/land/ad/4543673772?se=2menH1fS7hGArjBuXRw5xg&utm_medium=api&utm_so

TypeError: object of type 'JobOffer' has no len()

In [148]:
if "contract_type" in jobs_offers[-1]['results'][0]:
    print('este')
else:
    print('nope')

nope


La variable *jobs_offers* est un objet json donc type dictionnaire
on instancie *full_info* de la class GetJobOffer avec l'intégralité de *jobs_offers*

Les informations clés liés au poste se trouve dans *jobs_offers[results]* qui est une liste d'un seul élément, un dictionnaire dont chaque clé est une information utile 
- 'location' (elle même un dictionnaire)
    exemple 'location': {'__CLASS__': 'Adzuna::API::Response::Location', 'display_name': 'Toulouse, Haute-Garonne', 'area': ['France', 'Occitanie', 'Haute-Garonne', 'Toulouse']},
- 'category' (elle même un dictionnaire)
    exemple {'label': 'Unknown', '__CLASS__': 'Adzuna::API::Response::Category', 'tag': 'unknown'}
- title': 'Data Engineer PySpark'
- 'redirect_url': 'https://www.adzuna.fr/land/ad/4574739084?se=DGYosEfS7hGqg0-VTaQgFg&utm_medium=api&utm_source=37fe08af&v=FE7ECBBD26503585482CAD5270E4346D7E31A2D2'
- 'contract_time': 'full_time'
- 'contract_type': 'permanent'
- 'id': '4574739084'
- 'created': '2024-02-19T20:01:48Z'
- 'description'
    "QUI SOMMES-NOUS ? Thales propose des systèmes d'information et de communication sécurisés et interopérables pour les forces armées, les forces de sécurité et les opérateurs d'importance vitale. Ces activités, qui regroupent radiocommunications, réseaux, systèmes de protection, systèmes d'information critiques et cybersécurité, répondent aux besoins de marchés où l'utilisation des nouvelles technologies numériques est déterminante. Thales intervient tout au long de la chaîne de valeur, des équip…"
- 'salary_is_predicted': '0'
- 'company'(elle même un dictionnaire)
      {'display_name': 'THALES', '__CLASS__': 'Adzuna::API::Response::Company'}

In [152]:
df_adzuna_jobs = pd.DataFrame(jobs_ready_for_export)

df_adzuna_jobs.to_csv('output/job_offers_wttj.csv')
df_adzuna_jobs

ValueError: DataFrame constructor not properly called!